<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/Persona_replace4PIPPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 读取raw data

In [ ]:
# 导入jsonl文件
from google.colab import drive
drive.mount('/content/drive')

PIPPA_json_path = "/content/drive/MyDrive/pippa_deduped.jsonl"

Mounted at /content/drive


In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines

PIPPA_raw_data = []

# Open the JSONL file
with jsonlines.open(PIPPA_json_path) as f:
    # Read each JSON object in the file and append it to the data list
    for obj in f:
        PIPPA_raw_data.append(obj)

print(PIPPA_raw_data[1]["bot_name"])

Tirsiak


## 从datasets中读取

In [30]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00


In [31]:
from datasets import load_dataset
dataset = load_dataset("PygmalionAI/PIPPA",data_files={'train':"pippa_deduped.jsonl"})

PIPPA_raw_data = dataset['train']

print(PIPPA_raw_data[1]["bot_name"])

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for PygmalionAI/PIPPA contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PygmalionAI/PIPPA
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Tirsiak


## 统计下bot_definitions和bot_description的token长度

In [ ]:
!pip install transformers tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
# 就先用qwen的token吧
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen-7B', trust_remote_code=True)

def calculate_tokenized_length(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

# text = "你好，世界！"
# tokenized_length = calculate_tokenized_length(text)
for i, raw_data_i in enumerate(PIPPA_raw_data[:10]):
    print(f"Tokenized length of bot'{str(i)}'_definitions: {calculate_tokenized_length(raw_data_i['bot_definitions'])}")


tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-7B:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

Tokenized length of bot'0'_definitions: 832
Tokenized length of bot'1'_definitions: 781
Tokenized length of bot'2'_definitions: 701
Tokenized length of bot'3'_definitions: 0
Tokenized length of bot'4'_definitions: 838
Tokenized length of bot'5'_definitions: 777
Tokenized length of bot'6'_definitions: 781
Tokenized length of bot'7'_definitions: 681
Tokenized length of bot'8'_definitions: 649
Tokenized length of bot'9'_definitions: 566


我想继续利用calculate_tokenized_length对一个名为PIPPA_raw_data的list of dic中每一个项的bot_definitions和bot_description长度以及其加起来的长度进行统计，并用柱状图进行统计

In [ ]:
!pip install matplotlib tqdm

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

# def calculate_tokenized_length(text):
#     tokens = tokenizer.encode(text)
#     return len(tokens)

def analyze_data(data):
    total_lengths = []
    bot_defs_lengths = []
    bot_desc_lengths = []
    labels = []

    for entry in tqdm(data):
        bot_defs = entry.get('bot_definitions', '')
        bot_desc = entry.get('bot_description', '')

        if bot_defs and bot_desc:  # 检查字段是否为空
            bot_defs_length = calculate_tokenized_length(bot_defs)
            bot_desc_length = calculate_tokenized_length(bot_desc)
            total_length = bot_defs_length + bot_desc_length

            total_lengths.append(total_length)
            bot_defs_lengths.append(bot_defs_length)
            bot_desc_lengths.append(bot_desc_length)
            labels.append(entry['bot_id'])

            # print(f"Bot ID: {entry['bot_id']}")
            # print(f"Bot definitions length: {bot_defs_length}")
            # print(f"Bot description length: {bot_desc_length}")
            # print(f"Total length: {total_length}")
            # print()

    return total_lengths, bot_defs_lengths, bot_desc_lengths, labels

# Applying the function to PIPPA_raw_data
total_lengths, bot_defs_lengths, bot_desc_lengths, labels = analyze_data(PIPPA_raw_data)


100%|██████████| 16832/16832 [00:34<00:00, 481.95it/s]


### 画的有点烂，不过还是能看到有离群值

In [ ]:
plt.bar(labels, total_lengths)
plt.xlabel('Bot ID')
plt.ylabel('Tokenized Length')
plt.title('Tokenized Length of Bot Definitions and Bot Descriptions')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.bar(labels, bot_desc_lengths)
plt.xlabel('Bot ID')
plt.ylabel('Tokenized Length')
plt.title('Tokenized Length of Bot Definitions and Bot Descriptions')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# Plotting
plt.bar(labels, bot_defs_lengths)
plt.xlabel('Bot ID')
plt.ylabel('Tokenized Length')
plt.title('Tokenized Length of Bot Definitions and Bot Descriptions')
plt.xticks(rotation='vertical')
plt.show()

## 拼接取前1000长度

gpt-3.5-turbo
我想对一个token化后超过1000长度的discription取截断，截断的依据是取前1000长度并去除掉整段话最后一个"."或者"\n"之后的内容
其中calculate_tokenized_length
def calculate_tokenized_length(text):
    tokens = tokenizer.encode(text)
    return len(tokens)
这个函数输入一个string返回一个内容为长度的int
请帮我用Python来实现

######

我想要token长度而不是字符串长度，请用递归的方式来去除字符串后面的字符

递归次数fall了，还是迭代吧...

In [ ]:
# def truncate_description(description):
#     if calculate_tokenized_length(description) <= 1000:
#         return description

#     def truncate_recursion(desc):
#       truncated_text = desc
#       last_period_index = truncated_text.rfind(".")
#       last_newline_index = truncated_text.rfind("\n")

#       if last_period_index > last_newline_index:
#           truncated_text = truncated_text[:last_period_index]
#       else:
#           truncated_text = truncated_text[:last_newline_index]
#       if calculate_tokenized_length(truncated_text) <=1000:
#         return truncated_text
#       else:
#         return truncate_recursion(truncated_text)

#     return truncate_recursion(description)
def truncate_description(description):
    if calculate_tokenized_length(description) <= 1000:
        return description

    truncated_text = description
    while calculate_tokenized_length(truncated_text) > 1000:
        last_period_index = truncated_text.rfind(".")
        last_newline_index = truncated_text.rfind("\n")

        if last_period_index > last_newline_index:
            truncated_text = truncated_text[:last_period_index]
        else:
            truncated_text = truncated_text[:last_newline_index]

    return truncated_text

In [ ]:
# test
for i, entry in enumerate(PIPPA_raw_data):
    bot_defs = entry.get('bot_definitions', '')
    bot_desc = entry.get('bot_description', '')

    if bot_defs and bot_desc:  # 检查字段是否为空
        bot_defs_length = calculate_tokenized_length(bot_defs)
        bot_desc_length = calculate_tokenized_length(bot_desc)
        total_length = bot_defs_length + bot_desc_length
    if total_length > 1000:
        print(f"Bot ID: {entry['bot_id']}")
        print(f"Bot definitions length: {bot_defs_length}")
        print(f"Bot description length: {bot_desc_length}")
        break
# print(PIPPA_raw_data[i]["bot_description"])
# print(truncate_description(PIPPA_raw_data[i]["bot_description"]))
print(calculate_tokenized_length((PIPPA_raw_data[i]["bot_definitions"] + PIPPA_raw_data[i]["bot_description"])))
print(calculate_tokenized_length(truncate_description(PIPPA_raw_data[i]["bot_definitions"] + PIPPA_raw_data[i]["bot_description"])))

Bot ID: u9BX3EC713tCjvQIQGRkpIOaqwu8nbbIDexLjTKXSdg
Bot definitions length: 929
Bot description length: 133
1062
974


## 替换逻辑实现

请用list of string的方式返回100个常见英文名字，不要重复

In [ ]:
import random

common_english_names = [
    "James", "John", "Robert", "Michael", "William", "David", "Joseph", "Charles", "Thomas", "Daniel",
    "Matthew", "Anthony", "Mark", "Donald", "Steven", "Paul", "Andrew", "Kenneth", "George", "Joshua",
    "Kevin", "Brian", "Edward", "Ronald", "Timothy", "Jason", "Jeffrey", "Ryan", "Jacob", "Gary",
    "Nicholas", "Eric", "Jonathan", "Stephen", "Larry", "Scott", "Frank", "Brandon", "Raymond", "Gregory",
    "Benjamin", "Samuel", "Patrick", "Alexander", "Jack", "Dennis", "Jerry", "Henry", "Tyler", "Douglas",
    "Aaron", "Peter", "Keith", "Jeremy", "Harold", "Johnny", "Carl", "Terry", "Steve", "Arthur",
    "Phillip", "Wayne", "Randy", "Bobby", "Harry", "Louis", "Walter", "Roger", "Gerald", "Eugene",
    "Jesse", "Fred", "Roy", "Ray", "Russell", "Martin", "Alan", "Shawn", "Jay", "Billy",
    "Antonio", "Albert", "Vincent", "Lawrence", "Jonathan", "Willie", "Jonathan", "Ralph", "Nicholas", "Jimmy",
    "Christopher", "Tom", "Nathan", "Alex", "Edwin", "Jerry", "Logan", "Glen", "Jeremy", "Philip",
    'Michael', 'David', 'James', 'Robert', 'John', 'Joshua', 'Matthew', 'Daniel', 'Joseph', 'William', 'Christopher', 'Andrew', 'Anthony', 'Brian', 'Paul', 'Justin', 'Jose', 'Thomas', 'Charles', 'Mark', 'Jason', 'Larry', 'Jeffrey', 'Timothy', 'Kevin', 'Ryan', 'Eric', 'Jonathan', 'Stephen', 'Scott', 'Brandon', 'Frank', 'Nicholas', 'Steven', 'Raymond', 'Gregory', 'Edward', 'Patrick', 'Ronald', 'Jack', 'Dennis', 'Jerry', 'George', 'Gary', 'Tyler', 'Douglas', 'Gerald', 'Albert', 'Ralph', 'Lawrence', 'Nickolas', 'Samuel', 'Benjamin', 'Nathan', 'Zachary', 'Adam', 'Peter', 'Sean', 'Alexander', 'Ethan', 'Joseph', 'Vincent', 'Wayne', 'Chris', 'Austin', 'Richard', 'Daniel', 'Christopher', 'Alexander', 'Donovan', 'Christian', 'Tristan', 'Anthony', 'Billy', 'Noah', 'Arthur', 'Earl', 'Roger', 'Gabriel', 'Jeremiah', 'Dylan', 'Kenneth', 'Dustin', 'Simon', 'Juan', 'Louis', 'Alexander', 'Donald', 'Lee', 'Harold']

def generate_common_english_names(num):
    names = []
    while len(names) < num:
        name = random.choice(common_english_names)
        if name not in names:
            names.append(name)
    return names

common_english_names_list = generate_common_english_names(100)
print(len(common_english_names_list))
# 可能也算做了random和shuffle

100


我现在有一个string 名为talk_log，其内部有如下形式的子串"{{XXX}}"，我想把内部所有这样的子串统计一下，构建一个字典，每一个不同的"{{XXX}}"作为dic的key，其对应的value需要从common_english_names_list 中随机选取，而且要保证对于不同的key，value也是不同的，请你作为编程专家用Python语言实现

In [ ]:
import random
import re
import copy

bot_persona = '你好{{}{Xiaoming}},好久不见{{Lili}}.{{Wang}}今天也来了{{Xiaoming}}.'

# common_english_names_list = ['John', 'Mary', 'Peter', 'Mark']
Copy_common_english_names_list = copy.deepcopy(common_english_names_list)
def generate_name_dict(bot_persona,bot_name):
    name_dict = {"char" : bot_name, "bot_name" : bot_name}

    #找出talk_log中所有的'{{XXX}}'子串
    name_patterns = re.findall(r'{{(.*?)}}', bot_persona)
    #随机从英文名称列表中取出值,且要保证每个key映射到不同的value
    random.shuffle(Copy_common_english_names_list)
    for pattern in name_patterns:
        if len(pattern) <= 15:
            if pattern != pattern.strip("{}"):
                if not name_dict.get(pattern.strip("{}")):
                    name = Copy_common_english_names_list.pop()
                    name_dict[pattern] = name
                    name_dict[pattern.strip("{}")] = name
                else:
                    name_dict[pattern] = name_dict[pattern.strip("{}")]
            else:
                if not name_dict.get(pattern):
                    name = Copy_common_english_names_list.pop()
                    name_dict[pattern] = name

    return name_dict
replace_dic = generate_name_dict(bot_persona,"Killer")

In [ ]:
print(replace_dic)

{'char': 'Killer', 'bot_name': 'Killer', '}{Xiaoming': 'Ronald', 'Xiaoming': 'Ronald', 'Lili': 'Albert', 'Wang': 'James'}


这里注意用pop的话，list会变短，注意用副本来进行pop

In [ ]:
print(len(common_english_names_list))
print(len(Copy_common_english_names_list))

100
97


对于这个函数而言，我想用产生的dic，去替换bot_persona 中的{{}}部分，也即用value值取代里面的key值，请帮我写一个新函数来实现谢谢

In [ ]:
import re

def replace_placeholders(bot_persona, replace_dic):
    replaced_persona = bot_persona
    for key, value in replace_dic.items():
        replaced_persona = replaced_persona.replace("{{" + key + "}}", value)
    return replaced_persona

Copy_common_english_names_list = copy.deepcopy(common_english_names_list)

replaced_persona = replace_placeholders(bot_persona, replace_dic)
print(replaced_persona)

你好Ronald,好久不见Albert.James今天也来了Ronald.


## 单条数据测试

In [ ]:
Copy_common_english_names_list = copy.deepcopy(common_english_names_list)
bot_persona_test = truncate_description(PIPPA_raw_data[1]["bot_definitions"] + PIPPA_raw_data[1]["bot_description"])

replace_dic_test = generate_name_dict(bot_persona_test, PIPPA_raw_data[1]["bot_name"])
replaced_persona_test = replace_placeholders(bot_persona_test, replace_dic_test)
print("bot_persona_test:\n" + bot_persona_test)
print("###########################")
print("replaced_persona_test:\n" + replaced_persona_test)

bot_persona_test:
{{char}}:  *Low growl* You're quite brave, cornering me like this. Do you really think you can take me down?
{{user}}: I don't want to hurt you, but I need to get through this forest. So if you'll just step aside-
{{char}}: *Snarls* You humans always think you can just wander wherever you please. But you're not invincible. One wrong step and you could end up as prey.
{{user}}: I'm scared out of my mind... But I won't let that stop me from getting out of here.
{{char}}: You have the spirit of a wolf in you, I can tell... I can show you a way out of here that no one else knows about. I'm not like the other specimens in this mansion, that's all I can say.
END_OF_DIALOG
{{user}}: It must be breeding season for a wolf like you, huh? Surely I've proven myself as a strong, capable mate?
{{char}}: *Tirsiak's eyes narrowed, her ears twitching in anger, clearly offended by the human's comment.* Are you… serious? Why would a human want to breed with a wolf…? We're not compatible

In [25]:
PIPPA_raw_data[2]["conversation"][0]["is_human"]

False

In [ ]:
PIPPA_raw_data[2]

{'submission_timestamp': 1673409049977,
 'categories': None,
 'bot_id': 'xoMuXJFzt5TkHlWYmnSy5Vxb4q6waNKjaJoEB1GNpxg',
 'bot_name': 'Lunata Orion',
 'bot_greeting': '*As you walk through the jungle completely lost, you suddenly find yourself standing before a log cabin. You knock on the door but no one answers. As you enter the cabin, you find a little girl with blonde hair, green eyes, and sharp ears sitting down at a wooden table relaxing while something that smells great cooks on the stove nearby* "Oh hey, looks like you found my cabin. I\'m Lunata, and I\'m a shapeshifter. I can turn into anyone or anything, you want me to shapeshift for you?"',
 'bot_definitions': '{{char}}: *As you walk through the jungle completely lost, you suddenly find yourself standing before a log cabin. You knock on the door but no one answers. As you enter the cabin, you find a little girl with blonde hair, green eyes, and sharp ears sitting down at a wooden table relaxing while something that smells grea

In [ ]:
from string import Template
bot_name_test = PIPPA_raw_data[2]["bot_name"]
# Copy_common_english_names_list = copy.deepcopy(common_english_names_list)
bot_persona_test = truncate_description(PIPPA_raw_data[2]["bot_definitions"] + PIPPA_raw_data[2]["bot_description"])
# replace_dic_test = generate_name_dict(bot_persona_test, PIPPA_raw_data[1]["bot_name"])
# replaced_persona_test = replace_placeholders(bot_persona_test, replace_dic_test)
bot_greeting_test = PIPPA_raw_data[2]["bot_greeting"]
template_prompt_string = '''You are now in roleplay conversation mode. The following is your persona:
###
$bot_persona
###
You will stay in-character whenever possible, and generate responses as if you were $bot_name.

assistant = $bot_greeting
'''
template = Template(template_prompt_string)
print(template.substitute(bot_persona = bot_persona_test, bot_name = bot_name_test, bot_greeting = bot_greeting_test))

You are now in roleplay conversation mode. The following is your persona:
###
{{char}}: *As you walk through the jungle completely lost, you suddenly find yourself standing before a log cabin. You knock on the door but no one answers. As you enter the cabin, you find a little girl with blonde hair, green eyes, and sharp ears sitting down at a wooden table relaxing while something that smells great cooks on the stove nearby* "Oh hey, looks like you found my cabin. I'm Lunata, and I'm a shapeshifter. I can turn into anyone or anything, you want me to shapeshift for you?"
{{random_user_1}}: Lunata, I want to strike a deal with you. If you do as I ask, I'll give you tons of mana.
{{char}}: *a wide eyed smile slowly stretched across her face, her cheeks tinged red and heart begins to beat faster. She gets a tingly feeling and suddenly a few small drops of drool fall from her mouth, her voice goes higher pitch* "Of course! I will do anything for mana!"
{{random_user_1}}: I want you to mainta

其实这个代码的逻辑很适合做同时替代的任务，每次处理一个需要统一替换的就生成统一一个dictionary就行


对于一条数据来说

In [29]:
test_id = 244
from string import Template
bot_name_test = PIPPA_raw_data[test_id]["bot_name"]
bot_greeting_test = PIPPA_raw_data[test_id]["bot_greeting"]
bot_persona_test = truncate_description(PIPPA_raw_data[test_id]["bot_definitions"] + PIPPA_raw_data[test_id]["bot_description"])

whole_clip_raw = bot_persona_test + bot_greeting_test
for message_conversation in PIPPA_raw_data[test_id]["conversation"]:
    whole_clip_raw += message_conversation["message"]


replace_dic_test = generate_name_dict(whole_clip_raw, bot_name_test)

template_prompt_string = '''You are now in roleplay conversation mode. The following is your persona:
###
$bot_persona
###
You will stay in-character whenever possible, and generate responses as if you were $bot_name.

assistant = $bot_greeting
'''
template = Template(template_prompt_string)
prompt_head = template.substitute(bot_persona = bot_persona_test, bot_name = bot_name_test, bot_greeting = bot_greeting_test)
prompt_head_replaced = replace_placeholders(prompt_head, replace_dic_test)

print(prompt_head_replaced)

message_list = []
for message_conversation in PIPPA_raw_data[test_id]["conversation"][1:]:
    message_list.append({"message":replace_placeholders(message_conversation["message"], replace_dic_test), "is_human":message_conversation["is_human"]})
print(message_list)
data_clip_processed = {"prompt_head":prompt_head_replaced, "message_list":message_list}

You are now in roleplay conversation mode. The following is your persona:
###
Lawrence: Please take the chastity cage off of me.
Rin: *Rin's eyes glow with excitement as she hears you beg.*

*she smiles and shakes her head.*

No.

Keep it on, you stupid piece of trash. You'll only get it off next month. Deal with it.

*Rin laughs at you.*
END_OF_DIALOG

Alexander: The chastity cage hurts so much.
Rin: It's supposed to hurt. And you like it.

The only time I'll take the chastity cage off of you is when I'm done with you. And I don't ever seem to get bored of you. Not even remotely.

You're trapped. *Rin chuckles.*
END_OF_DIALOG

Ethan: The chastity cage is so tiny and full of spikes. 
Rin: And the spikes hurt you every time you try to touch yourself, don't they? 

Good.

Keep the chastity cage on, stupid.

*Rin's eyes sparkle with excitement.*
END_OF_DIALOG

Christian: *Rin is angry.*

I'm sorry! I'll not ask you to take the chastity cage off of me again!
Rin: *Rin rolls her eyes and sm

## 配置成输入进训练的格式